In [113]:
import time
from ipycytoscape import CytoscapeWidget
import networkx as nx
from IPython.display import clear_output
import random
import pandas as pd

In [128]:
f = pd.read_csv("viz_activity.csv",sep=",",names=['source','target','timestamp'])
d = pd.read_csv("viz_edgelist.csv",sep=",",names=['source','target'])

g = nx.from_pandas_edgelist(f,source="source",target="target",edge_attr = ["timestamp"])
g_l = list(g.edges(data=True))
g_l = sorted(g_l,key = lambda x: x[2]['timestamp'])

g_n = d.groupby('source').count().to_dict(orient='dict')['target']
for i in g_n.keys():
    g_n[i] = [g_n[i],"inactive",0,0,False,0,set(),-1,"grey",""]

In [129]:
g_n

{'a': [5, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'b': [4, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'c': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'd': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'e': [2, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'f': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'g': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'h': [2, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'i': [2, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'j': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'k': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', ''],
 'l': [1, 'inactive', 0, 0, False, 0, set(), -1, 'grey', '']}

In [130]:
g_l

[('a', 'c', {'timestamp': 1}),
 ('a', 'd', {'timestamp': 2}),
 ('a', 'e', {'timestamp': 3}),
 ('a', 'b', {'timestamp': 4}),
 ('e', 'i', {'timestamp': 5}),
 ('i', 'k', {'timestamp': 6}),
 ('b', 'f', {'timestamp': 7}),
 ('b', 'g', {'timestamp': 8}),
 ('b', 'h', {'timestamp': 9}),
 ('a', 'l', {'timestamp': 10}),
 ('h', 'j', {'timestamp': 11})]

In [131]:
graph = {'links' : g_l, 'nodes' : g_n}

In [132]:
def mod_graph(graph,ed):
    edges = []
    for i in range(len(ed)):
        edges.append({'data' : {'source' : d.iloc[i,:]['source'], 'target' : d.iloc[i,:]['target']}})
    
    nodes = []
    for i in graph['nodes'].keys():
        nodes.append({'data' : {'id' : i, 'color' : graph['nodes'][i][8], 'status' : graph['nodes'][i][9]}})
    
    data = {
        'nodes' : nodes,
        'edges' : edges
    }
    cyto = CytoscapeWidget()
    cyto.graph.add_graph_from_json(data)
    
    cyto.set_style([{
                        'selector': 'node',
                        'css': {
                            'content': 'data(status)',
                            'text-valign': 'center',
                            'color': 'white',
                            'text-outline-width': 1,
                            'text-outline-color': 'data(color)',
                            'background-color': 'data(color)'
                        }
                        },
                        {
                        'selector': ':selected',
                        'css': {
                            'background-color': 'black',
                            'line-color': 'black',
                            'target-arrow-color': 'black',
                            'source-arrow-color': 'black',
                            'text-outline-color': 'black'
                        }}
                        ])
    return cyto
    

In [133]:
def LT_Viz(graph,ed):
    active = []
    blocked = []
    count = 0
    for i in graph['links']:
        if graph['nodes'][i[0]][4] != True:
            graph['nodes'][i[0]][1] = "active"
            graph['nodes'][i[0]][2] = 15
            graph['nodes'][i[0]][3] = i[2]['timestamp']
            graph['nodes'][i[0]][8] = "green"
            
        for j in blocked:
            if graph['nodes'][j][5] - 1 == 0:
                graph['nodes'][j][4] = False
                graph['nodes'][j][5] = 0
                blocked.remove(j)
                if graph['nodes'][j][2] > 0:
                    graph['nodes'][j][8] = "green"
                else:
                    graph['nodes'][j][8] = "grey"
                cyto = mod_graph(graph,ed)
                display(cyto)
                time.sleep(2)
                clear_output(wait=True)
                graph['nodes'][j][9] = ""
            else:
                graph['nodes'][j][5] -= 1
        
        for j in active:
            if graph['nodes'][j][2] - 1 == 0:
                graph['nodes'][j][2] = 0
                graph['nodes'][j][1] = "inactive"
                graph['nodes'][j][8] = "grey"
                graph['nodes'][j][9] = "Inactive due to energy."
                active.remove(j)
                cyto = mod_graph(graph,ed)
                display(cyto)
                time.sleep(2)
                clear_output(wait=True)
                graph['nodes'][j][9] = "Inactive due to energy."
            else:
                graph['nodes'][j][2] -= 1
                if i[2]['timestamp'] - graph['nodes'][j][3] > 4:
                    graph['nodes'][j][1] = "inactive"
                    graph['nodes'][j][8] = "orange"
                    graph['nodes'][j][9] = "Node Minimizing Rumor."
                    active.remove(j)
                    cyto = mod_graph(graph,ed)
                    display(cyto)
                    time.sleep(2)
                    clear_output(wait=True)
                    graph['nodes'][j][9] = ""
                    r_node = graph['nodes'][j][7]
                    if graph['nodes'][r_node][4] == False:
                        temp_count = 0
                        for temp in graph['nodes'][r_node][6]:
                            if (graph['nodes'][temp][1] == "inactive" or (i[2]['timestamp'] - graph['nodes'][temp][3] > 3)) and graph['nodes'][temp][2] > 0:
                                temp_count += 1
                        if temp_count >= 0.6*len(graph['nodes'][r_node][6]):
                            graph['nodes'][r_node][4] = True
                            graph['nodes'][r_node][5] = 2
                            blocked.append(r_node)
                            graph['nodes'][r_node][8] = "red"
                            graph['nodes'][r_node][9] = "Node Blocked."
                            count += 1
                            cyto = mod_graph(graph,ed)
                            display(cyto)
                            time.sleep(2)
                            clear_output(wait=True)
                            graph['nodes'][r_node][9] = ""
                    else:
                        pass
                else:
                    pass
            
        graph['nodes'][i[0]][6].add(i[1])
        
        if graph['nodes'][i[0]][4] == True:
            graph['nodes'][i[0]][9] = "Sending."
            cyto = mod_graph(graph,ed)
            display(cyto)
            time.sleep(2)
            clear_output(wait=True)
            graph['nodes'][i[0]][9] = ""
            pass
          
        else:
            graph['nodes'][i[0]][9] = "Sending."
            graph['nodes'][i[1]][9] = "Receiving"
            cyto = mod_graph(graph,ed)
            display(cyto)
            time.sleep(2)
            clear_output(wait=True)
            graph['nodes'][i[0]][9] = ""
            graph['nodes'][i[1]][9] = ""
            if graph['nodes'][i[1]][1] == "inactive":
                if 1/graph['nodes'][i[0]][0] < 1/graph['nodes'][i[1]][0]:
                    graph['nodes'][i[1]][1] = "active"
                    graph['nodes'][i[1]][8] = "green"
                    graph['nodes'][i[1]][2] = 15
                    graph['nodes'][i[1]][3] = i[2]['timestamp']
                    graph['nodes'][i[1]][7] = i[0]
                    active.append(i[1])
                    graph['nodes'][i[1]][9] = "Activated"
                    cyto = mod_graph(graph,ed)
                    display(cyto)
                    time.sleep(2)
                    clear_output(wait=True)
                    graph['nodes'][i[1]][9] = ""       
            else:
                pass
        #print(graph['nodes'])

In [134]:
LT_Viz(graph,d)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…